- We are going to build an abstract base class that can't be instantiated, but instead, used as a base to build subclasses. The methods in an abstract class are only accessible by its subclasses.

- Each Constraint class will have variables and a method that checks whether the constraint is satisfied.

- To make an abstract class, we pass in __ABC__ from the module abc as a parameter and put the decorator __@abstractmethod__ on top of methods (other than init) of the abstract class.

- The abstract methods are inherited and we need to define them in full in our subclasses.

A quick video on abstract classes: https://www.youtube.com/watch?v=PDMe3wgAsWg

In [1]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    def __init__(self, variabls: List[V]) -> None:
        self.variables: variables
    
    # Must be overriden by subclass
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        pass

"The centerpiece of our constraint-satisfaction framework will be a class called __CSP__. __CSP__ is the gathering point for variables, domains, and constraints. In terms of its type hints, it uses generics to make itself flexible enough to work with any kind of variables and domain values (V keys and D domain values). Within __CSP__, the __variables__, __domains__, and __constraints__ collections are of types that you would expect. The __variables__ collection is a __list__ of variables, domains is a __dict__ mapping variables to lists of possible values (the domains of those variables), and __constraints__ is a __dict__ that maps each variable to a __list__ of the constraints imposed on it."

In [2]:
# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V,D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.constraints: Dict[V, List[Constraint[V,D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError('Every variable should have a domain assigned to it')
    
    def add_constraint(self, constraint: Constraint[V,D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError('Variable in constraint not in CSP')
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V,D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
            return True

We will use something called "backtracking" for our search. Backtracking is like depth-first search in that it goes back to the last known point where you made a decision before you hit a wall, and it choose a different path.

In [3]:
# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid
class CSP(Generic[V,D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.constraints: Dict[V, List[Constraint[V,D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError('Every variable should have a domain assigned to it')
    
    def add_constraint(self, constraint: Constraint[V,D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError('Variable in constraint not in CSP')
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V,D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
            return True
    ####### backtracking method ##########  
    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment
        
        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
            
        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy() # copy the assignment dictionary
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
            # if we can't find a result
            if result is not None:
                return result
        return None

These codes are saved in a file called csp.py

In [4]:
# testing to make sure the .py file works
import csp